In [1]:
import numpy as np
import gradio as gr
from keras.models import load_model
from PIL import Image
import cv2
import imutils

model = load_model("./Models/Custom_CNN(150x150).h5")

c:\Anaconda\envs\tgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels = [
    "glioma_tumor",
    "meningioma_tumor",
    "no_tumor",
    "pituitary_tumor",
]

def crop_images(image):
    image = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    if cnts:
        c = max(cnts, key=cv2.contourArea)
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = 0
        new_image = image[
            extTop[1] - ADD_PIXELS : extBot[1] + ADD_PIXELS,
            extLeft[0] - ADD_PIXELS : extRight[0] + ADD_PIXELS,
        ].copy()

        return new_image
    else:
        return None


def predict(image):
    cropped_image = crop_images(image)
    if cropped_image is None:
        return "No tumor detected", None
    cropped_image = np.array(cropped_image)
    cropped_image = cv2.resize(cropped_image, (150, 150))
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    cropped_image_input = np.expand_dims(cropped_image, axis=0)
    predictions = model.predict(cropped_image_input)[0]
    predicted_class = labels[np.argmax(predictions)]
    return predicted_class, cropped_image


image_input = gr.Image(type="pil", label="Original Image")
label_output = gr.Textbox(label="Predicted Class")
cropped_image_output = gr.Image(type="pil", label="Cropped Image")

interface = gr.Interface(
    fn=predict,
    inputs=image_input,
    outputs=[label_output, cropped_image_output],
    title="Brain Tumor Detection",
    description="Upload an image of a brain MRI scan to detect if there's a tumor.",
)

# Launch interface
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 4s 4s/step


In [ ]:
image_size = 150
path = "./Test2/"

In [ ]:
import os
import glob as gb
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm


def read_images_from_directory(
    folder_path,
    desc,
):
    images = []
    labels = []

    for folder in os.listdir(folder_path):
        files = gb.glob(pathname=str(folder_path + "/" + folder + "/*.jpg"))

        for file in tqdm(files, desc=desc + " in " + folder):
            image = plt.imread(file)
            img_resized = cv2.resize(image, (image_size, image_size))
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
            images.append(img_rgb)
            labels.append(folder)

    return images, labels

In [ ]:
x, y = read_images_from_directory(path, "Loading Data")

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer

x_data = np.array(x)
lb = LabelBinarizer()
y_data = lb.fit_transform(y)
print(y_data)

In [ ]:
from keras.models import load_model

model = load_model("./Models/ResNet101_(150x150).h5")

In [ ]:
test_loss, test_acc = model.evaluate(x_data, y_data)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

In [ ]:
y_pred_probs = model.predict(x_data)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_data, axis=1)

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    f1_score,
)

conf_matrix = confusion_matrix(y_true, y_pred)

accuracy = accuracy_score(y_true, y_pred)


precision = precision_score(y_true, y_pred, average="weighted")


recall = recall_score(y_true, y_pred, average="weighted")


f1 = f1_score(y_true, y_pred, average="weighted")

In [ ]:
print(f"Accuracy: {accuracy}")

print(f"Precision: {precision}")

print(f"Recall: {recall}")

print(f"F1 Score: {f1}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for the confusion matrix
plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=lb.classes_,
    yticklabels=lb.classes_,
    linewidths=0.5,
    linecolor="gray",
)

heatmap.set_xlabel("Predicted Labels", fontsize=14)
heatmap.set_ylabel("True Labels", fontsize=14)
heatmap.set_title("Confusion Matrix", fontsize=16)

plt.xticks(rotation=45, ha="right", fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.tight_layout()
plt.show()